In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN
import fasttext

import gensim
from gensim import corpora

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [4]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_predict

In [5]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [6]:
pd.set_option('display.max_columns', None)

### Retrieving the data

#### DF 2

In [7]:
df2 = pd.read_csv('df2.csv')

In [8]:
df2.head(2)

,Unnamed: 0,account_balance,duration,credit_history,purpose,credit_amount,savings_bond_value,employed_since,intallment_rate,sex_marital,guarantor,residence_since,property,age,other_installment_plans,type_of_housing,nr_of_existing_credits,job,number_of_dependents,telephone,foreign,target
0,0,< 0,6,critical account/other credits existing (not a...,radio/television,1169,unknown/no savings account,>=7 years,4,male;single,bank,4,real estate,67,none,own,2,skilled employee;official,1,Yes,Yes,1
1,1,0-200 DM,48,existing credits paid back duly till now,radio/television,5951,< 100 DM,1-4 years,2,female;divorced/separated/married,bank,2,real estate,22,none,own,1,skilled employee;official,1,No,Yes,2


# Models

In [9]:
# Initialize lists to store evaluation metrics for each fold
dataset_used = []
model_used = []
data_balancing_technique = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
conf_matrices = []

In [10]:
#combined_metrics = pd.DataFrame()

combined_metrics = pd.DataFrame(columns=['dataset', 'model', 'data balancing technique', 'fold', 'precision_1','precision_2','recall_1','recall_2','f1-score_1','f1-score_2','support_1','support_2','TP','FP','TN','FN'])

# Fast Text 
### train_supervised

## DF2

In [11]:
# Assuming 'df2' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df2.drop(columns=['target'])  # Drop the target column to get the feature columns
target = '__label__' + df2['target'].astype(str)

# Define hyperparameters
dim = 300  # Dimension of word vectors
min_count = 1  # Minimum frequency of words
loss = 'ns'  # Hierarchical softmax loss #'hs', 'ns', 'softmax'
epoch = 100  # Number of training epochs
bucket = 2000000  # Number of buckets used for hashing n-grams
word_ngrams = 1  # Maximum length of word n-grams
lr = 0.5 #0.1, 0.01, 0.001 #learning rate

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Create a new column "content" by concatenating values from selected columns
    X_train['content'] = y_train + ' ' + X_train.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w") as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    # Extract the numeric label using a lambda function
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF2',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'None',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.73
Mean Precision: 0.73
Mean Recall: 0.73
Mean F1-Score: 0.72


In [12]:
combined_metrics[(combined_metrics['data balancing technique'] == 'None') & (combined_metrics['dataset'] == 'DF2')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF2,Fast Text - train_supervised,None,1,0.833333,0.607143,0.845070,0.586207,0.839161,0.596491,71.0,29.0,60,12,17,11
1,DF2,Fast Text - train_supervised,None,2,0.788732,0.517241,0.800000,0.500000,0.794326,0.508475,70.0,30.0,56,15,15,14
2,DF2,Fast Text - train_supervised,None,3,0.772152,0.666667,0.897059,0.437500,0.829932,0.528302,68.0,32.0,61,18,14,7
3,DF2,Fast Text - train_supervised,None,4,0.817073,0.666667,0.917808,0.444444,0.864516,0.533333,73.0,27.0,67,15,12,6
4,DF2,Fast Text - train_supervised,None,5,0.768116,0.419355,0.746479,0.448276,0.757143,0.433333,71.0,29.0,53,16,13,18
5,DF2,Fast Text - train_supervised,None,6,0.693333,0.600000,0.838710,0.394737,0.759124,0.476190,62.0,38.0,52,23,15,10
6,DF2,Fast Text - train_supervised,None,7,0.810811,0.346154,0.779221,0.391304,0.794702,0.367347,77.0,23.0,60,14,9,17
7,DF2,Fast Text - train_supervised,None,8,0.830986,0.689655,0.867647,0.625000,0.848921,0.655738,68.0,32.0,59,12,20,9
8,DF2,Fast Text - train_supervised,None,9,0.812500,0.500000,0.866667,0.400000,0.838710,0.444444,75.0,25.0,65,15,10,10
9,DF2,Fast Text - train_supervised,None,10,0.763889,0.642857,0.846154,0.514286,0.802920,0.571429,65.0,35.0,55,17,18,10


### FastText train_supervised with df2 data and imbalance data tackling (RandomUnderSampler)

In [13]:
# Assuming 'df2' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df2.drop(columns=['target'])  # Drop the target column to get the feature columns
target = '__label__' + df2['target'].astype(str)

# Define hyperparameters
dim = 300  # Dimension of word vectors
min_count = 1  # Minimum frequency of words
loss = 'ns'  # Hierarchical softmax loss #'hs', 'ns', 'softmax'
epoch = 100  # Number of training epochs
bucket = 2000000  # Number of buckets used for hashing n-grams
word_ngrams = 1  # Maximum length of word n-grams
lr = 0.5 #0.1, 0.01, 0.001 #learning rate

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Oversampling the minority class using SMOTE
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    
    # Create a new column "content" by concatenating values from selected columns
    X_train_resampled['content'] = y_train_resampled + ' ' + X_train_resampled.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train_resampled['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w") as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    # Extract the numeric label using a lambda function
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF2',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'Random Under Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.69
Mean Precision: 0.72
Mean Recall: 0.69
Mean F1-Score: 0.70


In [14]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Under Sampler') & (combined_metrics['dataset'] == 'DF2')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
10,DF2,Fast Text - train_supervised,Random Under Sampler,1,0.851852,0.456522,0.647887,0.724138,0.736000,0.560000,71.0,29.0,46,8,21,25
11,DF2,Fast Text - train_supervised,Random Under Sampler,2,0.824561,0.465116,0.671429,0.666667,0.740157,0.547945,70.0,30.0,47,10,20,23
12,DF2,Fast Text - train_supervised,Random Under Sampler,3,0.847458,0.560976,0.735294,0.718750,0.787402,0.630137,68.0,32.0,50,9,23,18
13,DF2,Fast Text - train_supervised,Random Under Sampler,4,0.819672,0.410256,0.684932,0.592593,0.746269,0.484848,73.0,27.0,50,11,16,23
14,DF2,Fast Text - train_supervised,Random Under Sampler,5,0.814815,0.413043,0.619718,0.655172,0.704000,0.506667,71.0,29.0,44,10,19,27
15,DF2,Fast Text - train_supervised,Random Under Sampler,6,0.682927,0.423729,0.451613,0.657895,0.543689,0.515464,62.0,38.0,28,13,25,34
16,DF2,Fast Text - train_supervised,Random Under Sampler,7,0.841270,0.351351,0.688312,0.565217,0.757143,0.433333,77.0,23.0,53,10,13,24
17,DF2,Fast Text - train_supervised,Random Under Sampler,8,0.851852,0.521739,0.676471,0.750000,0.754098,0.615385,68.0,32.0,46,8,24,22
18,DF2,Fast Text - train_supervised,Random Under Sampler,9,0.847458,0.390244,0.666667,0.640000,0.746269,0.484848,75.0,25.0,50,9,16,25
19,DF2,Fast Text - train_supervised,Random Under Sampler,10,0.823529,0.530612,0.646154,0.742857,0.724138,0.619048,65.0,35.0,42,9,26,23


### FastText train_supervised with df2 data and imbalance data tackling (RandomOverSampler)

In [15]:
# Assuming 'df2' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df2.drop(columns=['target'])  # Drop the target column to get the feature columns
target = '__label__' + df2['target'].astype(str)

# Define hyperparameters
dim = 300  # Dimension of word vectors
min_count = 1  # Minimum frequency of words
loss = 'ns'  # Hierarchical softmax loss #'hs', 'ns', 'softmax'
epoch = 100  # Number of training epochs
bucket = 2000000  # Number of buckets used for hashing n-grams
word_ngrams = 1  # Maximum length of word n-grams
lr = 0.5 #0.1, 0.01, 0.001 #learning rate

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Oversampling the minority class using SMOTE
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Create a new column "content" by concatenating values from selected columns
    X_train_resampled['content'] = y_train_resampled + ' ' + X_train_resampled.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train_resampled['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w") as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    # Extract the numeric label using a lambda function
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF2',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'Random Over Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.70
Mean Precision: 0.72
Mean Recall: 0.70
Mean F1-Score: 0.70


In [16]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Over Sampler') & (combined_metrics['dataset'] == 'DF2')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
20,DF2,Fast Text - train_supervised,Random Over Sampler,1,0.826087,0.548387,0.802817,0.586207,0.814286,0.566667,71.0,29.0,57,12,17,14
21,DF2,Fast Text - train_supervised,Random Over Sampler,2,0.803030,0.500000,0.757143,0.566667,0.779412,0.531250,70.0,30.0,53,13,17,17
22,DF2,Fast Text - train_supervised,Random Over Sampler,3,0.773333,0.600000,0.852941,0.468750,0.811189,0.526316,68.0,32.0,58,17,15,10
23,DF2,Fast Text - train_supervised,Random Over Sampler,4,0.813333,0.520000,0.835616,0.481481,0.824324,0.500000,73.0,27.0,61,14,13,12
24,DF2,Fast Text - train_supervised,Random Over Sampler,5,0.776119,0.424242,0.732394,0.482759,0.753623,0.451613,71.0,29.0,52,15,14,19
25,DF2,Fast Text - train_supervised,Random Over Sampler,6,0.661765,0.468750,0.725806,0.394737,0.692308,0.428571,62.0,38.0,45,23,15,17
26,DF2,Fast Text - train_supervised,Random Over Sampler,7,0.833333,0.392857,0.779221,0.478261,0.805369,0.431373,77.0,23.0,60,12,11,17
27,DF2,Fast Text - train_supervised,Random Over Sampler,8,0.840580,0.677419,0.852941,0.656250,0.846715,0.666667,68.0,32.0,58,11,21,10
28,DF2,Fast Text - train_supervised,Random Over Sampler,9,0.845070,0.482759,0.800000,0.560000,0.821918,0.518519,75.0,25.0,60,11,14,15
29,DF2,Fast Text - train_supervised,Random Over Sampler,10,0.768116,0.612903,0.815385,0.542857,0.791045,0.575758,65.0,35.0,53,16,19,12


## Performance metrics for the Linear Regression

In [17]:
combined_metrics

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF2,Fast Text - train_supervised,None,1,0.833333,0.607143,0.845070,0.586207,0.839161,0.596491,71.0,29.0,60,12,17,11
1,DF2,Fast Text - train_supervised,None,2,0.788732,0.517241,0.800000,0.500000,0.794326,0.508475,70.0,30.0,56,15,15,14
2,DF2,Fast Text - train_supervised,None,3,0.772152,0.666667,0.897059,0.437500,0.829932,0.528302,68.0,32.0,61,18,14,7
3,DF2,Fast Text - train_supervised,None,4,0.817073,0.666667,0.917808,0.444444,0.864516,0.533333,73.0,27.0,67,15,12,6
4,DF2,Fast Text - train_supervised,None,5,0.768116,0.419355,0.746479,0.448276,0.757143,0.433333,71.0,29.0,53,16,13,18
5,DF2,Fast Text - train_supervised,None,6,0.693333,0.600000,0.838710,0.394737,0.759124,0.476190,62.0,38.0,52,23,15,10
6,DF2,Fast Text - train_supervised,None,7,0.810811,0.346154,0.779221,0.391304,0.794702,0.367347,77.0,23.0,60,14,9,17
7,DF2,Fast Text - train_supervised,None,8,0.830986,0.689655,0.867647,0.625000,0.848921,0.655738,68.0,32.0,59,12,20,9
8,DF2,Fast Text - train_supervised,None,9,0.812500,0.500000,0.866667,0.400000,0.838710,0.444444,75.0,25.0,65,15,10,10
9,DF2,Fast Text - train_supervised,None,10,0.763889,0.642857,0.846154,0.514286,0.802920,0.571429,65.0,35.0,55,17,18,10


In [18]:
# Assuming df is your DataFrame
combined_metrics.to_csv('Output Data/Fast Text - train_supervised.csv')